### Import Libraries and IBM PAIRS Authentication

In [1]:
import pandas as pd
from ibmpairs import paw, authentication
import json
import numpy
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
import re
import numpy as np
import traceback
import rasterio
import os
import zipfile
import traceback
import zipfile
import glob

In [2]:
logging.basicConfig(level=logging.INFO)
pawLogger = logging.getLogger('ibmpairs.paw')
pawLogger.setLevel(logging.INFO)

In [3]:
with open("ibmpairskey.txt", "r") as f:
    pairs_key = f.read()

In [4]:
def authenticate(pairs_key):
    
    pairs_credentials = authentication.OAuth2(api_key=pairs_key)
    auth_header = {'Authorization': f'Bearer {pairs_credentials.jwt_token}'}
    PAIRS_SERVER   = 'https://pairs.res.ibm.com'
    PAIRS_CREDENTIALS = authentication.OAuth2(api_key=pairs_key)
    
    return PAIRS_SERVER, PAIRS_CREDENTIALS

### Define Parameters
Define bbox coordinates,start and end dates, query structure, layers

In [5]:
# Sentinel 2 Layers
senLayers = {
    "Red" : "49360",
    "Green" : "49681",
    "Blue" : "49680",
    "NIR": "49361",
    "NDVI" : "49464",
    "SCL": "49362",
    "B8a": "49685", #Narrow IR
    "B12": "49687", # SWIR 2200 nm
    "B11" : "49686", # SWIR 1610 nm
    "B5" : "49682", # vegetation red edge
    "B6" : "49683", # vegetation red edge
    "B7" : "49684", # vegetation red edge
    "CP" : "50250" # cloud prob map
}

In [6]:
# Extra derived layers for Sentinel 2

index_layers = [
    {
    "alias": "NDWI",
    "expression": "($Green - $NIR)/($Green + $NIR)"
    },
    {
    "alias": "NDMI",
    "expression": "($NIR - $B11)/($NIR + $B11)"
    }, 
    {
    "alias": "SAVI",
    "expression": "($NIR - $Red)/(($NIR + $Red + 0.5)*(1.0 + 0.5))"
    }, 
    {
    "alias": "ARVI",
    "expression": "($B8a - $Red -0.106*($Red - $Blue))/($B8a +$Red - 0.106*($Red - $Blue))"
    },
    {
    "alias": "MSI",
    "expression": "$B11/$B8a"
    },
    {
    "alias": "NDVIre",
    "expression": "($B8a-$B5)/($B8a+$B5)"
    }    
]

In [7]:
# ERA Layers
eraLayers = {
    "Red" : "49360",
    "Total Prec" : "49459",
    "SP" : "49439",
    "Temp" : "49423",
    "AWVC" : "49458",
    #"PrecType" : "49435", #skip for now since mode aggregation is not possible
    "SR" : "49440",
    "TCC" : "49454",
    "MinT" : "49429",
    "MaxT" : "49430"
}

In [8]:
# Using Beni's method (approach 1)

vpd_layers = [
        {
            #convert kelvin to celcius
            "alias": "tmaxc",
            "expression": "$MaxT - 273.15",
             "output" : "false"
        },
        {            
            # Min Rel humidity
            "type" : "raster", "id" : "48760",
            "aggregation" : "Min",
            "alias": "relativehum",
            "output" : "false"
        },
        {
            # calculate saturation water vapour pressure in Pa
            "alias": "esat",
            "expression": "611*math:exp((17.27 * $tmaxc)/($tmaxc + 237.3))",
            "output" : "false"
        },
        {
            # Calculate vpd
            "alias": "vpd",
            "expression": "$esat * (1 - $relativehum)" #,
            #"output" : "false"
        },
                {
            # Calculate vpd
            "alias": "vpd_final",
            "expression": "math:max(0, $vpd)"
        }
]

In [9]:
# Define date and space parameters
date_start, date_end = None, None #Needed for defining the query for ERA5
date = None #Needed for defining the query for Sentinel 2
bbox = None

In [10]:
# Define areas
areas = {
    'bbox_1' : ["46.94599", "7.68597", "47.20599", "7.94597"]
    #'bbox_2' : ["47.20599","7.94597","47.46599","8.20597"]
}

In [11]:
# Testing the max area

areas = {
    'bbox_1' : ["47.1", "6.9", "47.15", "7.95"]
    #'bbox_2' : ["47.20599","7.94597","47.46599","8.20597"]
}

# 0.9 deg = 8.8 GB for Sen2, 7.5 GB for ERA5
# 1 deg = 10.9 GB for Sen2, 8.84 for ERA5

In [12]:
# Define the query for Sentinel 2
querySentinelJson = {
    "layers" : [
        {
            
            "type" : "raster", "id" : senLayers[lKey], "alias": lKey
        }
        for lKey in senLayers
    ] + index_layers,
    "spatial" : {"type" : "square",  "coordinates" : bbox }, 
    "temporal" : {"intervals" : [{"snapshot" : date}]},
    "processor" : [{
        "order" : 1,
        "type" : "coarse-grain",
        "options" : [
            {"name" : "levelsUp", "value" : "2"},
            {"name" : "aggregation", "value" : "bilinear"}
        ]
    }] 
}

In [13]:
# Define the query for ERA5
queryEraJson = {
    "layers" : [
        {            
            "type" : "raster", "id" : eraLayers[lKey], "alias": lKey,
            "aggregation" : None
        }
        for lKey in eraLayers    
    ] + vpd_layers,
    "spatial" : {"type" : "square",  "coordinates" : bbox }, 
    "temporal" : {"intervals" : [
      {"start" : date_start, "end" : date_end}
  ]},
    "processor" : [{
        "order" : 1,
        "type" : "coarse-grain",
        "options" : [
            {"name" : "levelsUp", "value" : "2"},
            {"name" : "aggregation", "value" : "bilinear"}
        ]
    }]
}

In [14]:
# Define   aggregation type for each ERA 5 layer
agg_mean = ["49360", "49439", "49454", "49423", "49458"] #Red, Surface pressure, Total cloud cover, Temperature, AWVC
agg_sum = ["49459", "49440"] # Total precipitation, Solar radiation
agg_max = ["49430"] # Max temperature
agg_min = ["49429"] # Min temperature
#agg_mode = ["49435"] # Prec type
# still need to compute Vapor Pressure Deficit

In [15]:
for idx, layer in enumerate(queryEraJson["layers"]):
    if "id" in layer:
        if layer["id"] in agg_mean:
            queryEraJson["layers"][idx]["aggregation"] = 'Mean' #'Median' doesnt exist
        if layer["id"] in agg_sum:
            queryEraJson["layers"][idx]["aggregation"] = 'Sum'
        if layer["id"] in agg_max:
            queryEraJson["layers"][idx]["aggregation"] = 'Max'
        if layer["id"] in agg_min:
            queryEraJson["layers"][idx]["aggregation"] = 'Min'
    else:
        continue

### Download Data
Loop through grid coordinates and only download ERA5 when Sentinel 2 is available

In [16]:
# Function that sets the date range
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [17]:
from datetime import date
# Define start and end date
start_date = date(2019, 4, 5)
end_date = date(2019, 4, 6)

iso8601 = '%Y-%m-%dT%H:%M:%SZ'

In [19]:
# Authenticate before every query because of the time-out problem
for bbox, coord in areas.items():
    
    for single_date in daterange(start_date, end_date):
        print(f'Trying for date {single_date}')
        PAIRS_SERVER, PAIRS_CREDENTIALS = authenticate(pairs_key)
    
        # Try Sentinel 2 first
    
        date = single_date.strftime(iso8601)
        querySentinelJson["temporal"]["intervals"][0]["snapshot"] = date
        querySentinelJson["spatial"]["coordinates"] = coord
        sentinel_check = 0 # to check if there is Sentinel 2 data for this date or not
        
        try:    
            querySentinel = paw.PAIRSQuery(querySentinelJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
            querySentinel.submit()
        
            querySentinel.poll_till_finished()
            querySentinel.download()
            querySentinel.create_layers()
            
            #Check if the Sentinel folder exists
            path_sentinel = 'downloads/SENTINEL 2/'
            isExistSentinel = os.path.exists(path_sentinel)                          
            
            if isExistSentinel == True : 
                pass
            else:
                os.makedirs(path_sentinel)
                
            
            #Rename File
            old_name = 'downloads/' + str(querySentinel.zipFilePath)
            new_name = 'downloads/SENTINEL 2/' + str(date) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3]) + '.zip'
            os.rename(old_name, new_name)
            
            #extract zip file
            directory_name = 'downloads/SENTINEL 2/' + str(date) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3])
            with zipfile.ZipFile(new_name, 'r') as zip_ref:
                zip_ref.extractall(directory_name)
            os.remove(new_name)
            
            #remove original files
            for f in os.listdir(directory_name + "/original/"):
                os.remove(directory_name + "/original/" + f)
            os.rmdir(directory_name + "/original/")
            
            print('Sentinel-2 data downloaded')
            
            # Could add some data manipulation here and then save the data
            sentinel_check = 1
        except:
            print('No Sentinel-2 data for this date') 
       
    
        if sentinel_check:
            # Now download ERA 5 for dates of Sentinel 2
            date_start = single_date.strftime(iso8601)
            date_end = (single_date + timedelta(1)).strftime(iso8601)
            queryEraJson["temporal"]["intervals"][0]["start"] = date_start
            queryEraJson["temporal"]["intervals"][0]["end"] = date_end
            queryEraJson["spatial"]["coordinates"] = coord


            try:    
                queryEra = paw.PAIRSQuery(queryEraJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
                queryEra.submit()

                queryEra.poll_till_finished()
                queryEra.download()
                queryEra.create_layers()
                
                #Check if the ERA folder exists
                path_era = 'downloads/ERA5/'
                isExistEra = os.path.exists(path_era)                          
            
                if isExistEra == True : 
                    pass
                else:
                    os.makedirs(path_era)
                
                #Rename File
                old_name = 'downloads/' + str(queryEra.zipFilePath)
                new_name = 'downloads/ERA5/' + str(date) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3]) + '.zip'
                os.rename(old_name, new_name)
                
                        
                # Delete Sentinel 2 files in ERA folder            
                #extract zip file
                directory_name = 'downloads/ERA5/' + str(date) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3])
                with zipfile.ZipFile(new_name, 'r') as zip_ref:
                    zip_ref.extractall(directory_name)

                #delete the zip file
                os.remove(new_name)

                #remove Sentinel 2 bands from the directory
                for f in glob.glob(directory_name + "/High*.*"):
                    os.remove(f)
                #remove original files
                for f in os.listdir(directory_name + "/original/"):
                    os.remove(directory_name + "/original/" + f)
                os.rmdir(directory_name + "/original/")
                
                print('ERA-5 data downloaded')
                
                
            except:
                print('No ERA 5 data for this date')
  

Trying for date 2019-04-05


INFO:ibmpairs.paw:PAIRS query JSON initialized.
INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1666756800_27165169
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1666756800_27165169_4823d16415989c29e3196162fd201669.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.
INFO:ibmpairs.paw:PAIRS query JSON initialized.


Sentinel-2 data downloaded


INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1666756800_27776641
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1666756800_27776641_476182c307777bfeb74ac907675f7874.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.


ERA-5 data downloaded


In [ ]:
# DOWNLOADING ALL DATES ERA 5: add red to both Sen2 and ERA5

# Authenticate before every query because of the time-out problem
for bbox, coord in areas.items():
    
    for single_date in daterange(start_date, end_date):
        print(f'Trying for date {single_date}')
        PAIRS_SERVER, PAIRS_CREDENTIALS = authenticate(pairs_key)
    
        # Try Sentinel 2 first
    
        date = single_date.strftime(iso8601)
        querySentinelJson["temporal"]["intervals"][0]["snapshot"] = date
        querySentinelJson["spatial"]["coordinates"] = coord
        sentinel_check = 0 # to check if there is Sentinel 2 data for this date or not
    
        try:   
            print(f'Trying Sentinel...')
            querySentinel = paw.PAIRSQuery(querySentinelJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
            querySentinel.submit()
        
            querySentinel.poll_till_finished()
            querySentinel.download()
            querySentinel.create_layers()
            
            #Check if the Sentinel folder exists
            path_sentinel = 'downloads/SENTINEL 2/'
            isExistSentinel = os.path.exists(path_sentinel)                          
            
            if isExistSentinel == True : 
                pass
            else:
                os.makedirs(path_sentinel)
                
            
            #remove original files
                for f in os.listdir(directory_name + "/original/"):
                    os.remove(f)
                os.rmdir(directory_name + "/original/")
            
            # Could add some data manipulation here and then save the data
            sentinel_check = 1
        except:
            print('No Sentinel-2 data for this date')
        
    

        # Now download ERA 5
        date_start = single_date.strftime(iso8601)
        date_end = (single_date + timedelta(1)).strftime(iso8601)
        queryEraJson["temporal"]["intervals"][0]["start"] = date_start
        queryEraJson["temporal"]["intervals"][0]["end"] = date_end
        queryEraJson["spatial"]["coordinates"] = coord
        
    

        try:   
            print(f'Trying ERA...')
            queryEra = paw.PAIRSQuery(queryEraJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
            queryEra.submit()

            queryEra.poll_till_finished()
            queryEra.download()
            queryEra.create_layers()

            #Check if the ERA folder exists
            path_era = 'downloads/ERA5/'
            isExistEra = os.path.exists(path_era)                          

            if isExistEra == True : 
                pass
            else:
                os.makedirs(path_era)

            #Rename File
            old_name = 'downloads/' + str(queryEra.zipFilePath)
            new_name = 'downloads/ERA5/' + str(date) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3]) + '.zip'
            os.rename(old_name, new_name)

            # Delete Sentinel 2 files             
            #extract zip file
            directory_name = 'downloads/ERA5/' + str(date) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3])
            with zipfile.ZipFile(new_name, 'r') as zip_ref:
                zip_ref.extractall(directory_name)

            #delete the zip file
            os.remove(new_name)

            #remove Sentinel 2 bands from the directory
            for f in glob.glob(directory_name + "/High*.*"):
                os.remove(f)
            #remove original files
            for f in os.listdir(directory_name + "/original/"):
                os.remove(f)
            os.rmdir(directory_name + "/original/")
            
            print('ERA-5 data downloaded')

        except:
            print('No ERA 5 data for this date')

### Test removing file from zip

In [ ]:
# Remove files from ERA5 zip
import subprocess
cmd = ['zip', '-d', new_name] + moved_files
subprocess.check_call(cmd)
subprocess.run(cmd)

In [ ]:
import subprocess
import zipfile

z = zipfile.ZipFile(zip_filename)

files_to_del = filter( lambda f: f.endswith('exe'), z.namelist()]

cmd=['zip', '-d', zip_filename] + files_to_del
subprocess.check_call(cmd)

# reload the modified archive
z = zipfile.ZipFile(zip_filename)

In [ ]:
zf.namelist()

# Download Landcover and DEM

In [5]:
landcoverID = '51353'
demID = '49506'
redID = '49360'

In [6]:
# Define areas
areas = {
    'bbox_1' : ["46.94599", "7.68597", "47.20599", "7.94597"],
    'bbox_2' : ["47.20599","7.94597","47.46599","8.20597"]
}

In [7]:
areas = {
    'bbox_1' : ["46", "7", "46.5", "7.5"]
}

### Download DEM

In [8]:
from datetime import date

date_dem = datetime(2013, 1, 1, 12)
date_red = date(2016,8,3)
iso8601 = '%Y-%m-%dT%H:%M:%SZ'

bbox = None

In [9]:
# Define the query for DEM --> no time dependence

queryDemJson = {
    'layers' : [
        {            
            'type' : 'raster', 'id' : demID,
            "temporal" : {"intervals" : [{"snapshot" : date_dem.strftime(iso8601)}]} 
        },
        {   
            'type' : 'raster', 'id' : redID,
            "temporal" : {"intervals" : [{"snapshot" : date_red.strftime(iso8601)}]}
            #, 'output' : False
        }
    ],
    "spatial" : {"type" : "square",  "coordinates" : bbox },
    'temporal' : {'intervals' : [{'snapshot' : date_dem.strftime(iso8601)}]}
}

In [21]:
# Download DEM data

for bbox, coord in areas.items():
    
    PAIRS_SERVER, PAIRS_CREDENTIALS = authenticate(pairs_key)
    queryDemJson["spatial"]["coordinates"] = coord
    print(queryDemJson)

    try:    
        queryDem = paw.PAIRSQuery(queryDemJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
        queryDem.submit()

        queryDem.poll_till_finished()
        queryDem.download()
        queryDem.create_layers()

        #Check if the Sentinel folder exists
        path_Dem = 'downloads/NASA DEM/'
        isExistDem = os.path.exists(path_Dem)                          

        if isExistDem == True : 
            pass
        else:
            os.makedirs(path_Dem)

        #Rename File
        old_name = 'downloads/' + str(queryDem.zipFilePath)
        new_name = 'downloads/NASA DEM/' + str(year) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3]) + '.zip'
        directory_name = 'downloads/NASA DEM/' + str(year) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3])
        os.rename(old_name, new_name)
        
        #extract zip file
        with zipfile.ZipFile(new_name, 'r') as zip_ref:
            zip_ref.extractall(directory_name)
        
        #delete the zip file
        os.remove(new_name)
        
        #remove Sentinel 2 bands from the directory
        for f in glob.glob(directory_name + "/High*.*"):
            os.remove(f)
        
        print('NASA DEM data downloaded')

    except:
        traceback.print_exc()

INFO:ibmpairs.paw:PAIRS query JSON initialized.


{'layers': [{'type': 'raster', 'id': '49506', 'temporal': {'intervals': [{'snapshot': '2013-01-01T12:00:00Z'}]}}, {'type': 'raster', 'id': '49360', 'temporal': {'intervals': [{'snapshot': '2016-08-03T00:00:00Z'}]}}], 'spatial': {'type': 'square', 'coordinates': ['46', '7', '46.5', '7.5']}, 'temporal': {'intervals': [{'snapshot': '2013-01-01T12:00:00Z'}]}}


INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1664856000_32156214
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1664856000_32156214_510f13335410fbd2c2d1bec18e765dcd.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.


NASA DEM data downloaded


### Download Landcover

In [11]:
# Define date and space parameters
from datetime import date

landcover_years = [date(2016, 1, 1), date(2017, 1, 1), date(2018, 1, 1), date(2019, 1, 1)]
date_red = date(2016,8,3)
date = None
iso8601 = '%Y-%m-%dT%H:%M:%SZ'

bbox = None

In [12]:
queryLCJson = {
    'layers' : [
        {            
            'type' : 'raster', 'id' : landcoverID,
            "temporal" : {"intervals" : [{"snapshot" : date}]} 
        },
        {   
            'type' : 'raster', 'id' : redID,
            "temporal" : {"intervals" : [{"snapshot" : date_red.strftime(iso8601)}]}
            #, 'output' : False
        }
    ],
    "spatial" : {"type" : "square",  "coordinates" : bbox },
    'temporal' : {'intervals' : [{'snapshot' : date }]}
}

In [22]:
# Download Landcover data

for bbox, coord in areas.items():
    for year in landcover_years:

        print(f'Trying for date {year}')
        PAIRS_SERVER, PAIRS_CREDENTIALS = authenticate(pairs_key)

        date = year.strftime(iso8601)
        queryLCJson["layers"][0]["temporal"]["intervals"][0]["snapshot"] = date
        queryLCJson["temporal"]["intervals"][0]["snapshot"] = date
        queryLCJson["spatial"]["coordinates"] = coord

        
        try:    
            queryLC = paw.PAIRSQuery(queryLCJson, PAIRS_SERVER, PAIRS_CREDENTIALS, authType='api-key', overwriteExisting=False)
            queryLC.submit()

            queryLC.poll_till_finished()
            queryLC.download()
            queryLC.create_layers()

            #Check if the Sentinel folder exists
            path_LC = 'downloads/LC Copernicus/'
            isExistLC = os.path.exists(path_LC)                          

            if isExistLC == True : 
                pass
            else:
                os.makedirs(path_LC)

            #Rename File
            old_name = 'downloads/' + str(queryLC.zipFilePath)
            new_name = 'downloads/LC Copernicus/' + str(year) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3]) + '.zip'
            directory_name = 'downloads/LC Copernicus/' + str(year) + '_' + str(coord[0]) + '_' + str(coord[1]) + '_' + str(coord[2]) + '_' + str(coord[3])
            os.rename(old_name, new_name)
            
             #extract zip file
            with zipfile.ZipFile(new_name, 'r') as zip_ref:
                zip_ref.extractall(directory_name)
        
            #delete the zip file
                os.remove(new_name)
        
            #remove Sentinel 2 bands from the directory
            for f in glob.glob(directory_name + "/High*.*"):
                os.remove(f)
            
            print('Landcover data downloaded')
            
        except:
            print('No Landcover data for this year')

Trying for date 2016-01-01


INFO:ibmpairs.paw:PAIRS query JSON initialized.
INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1664856000_32467882
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1664856000_32467882_381c71d2f8e1a8c64fcca39a8a07124e.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.


Landcover data downloaded
Trying for date 2017-01-01


INFO:ibmpairs.paw:PAIRS query JSON initialized.
INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1664856000_32555880
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1664856000_32555880_39323cf3574657f69667f4b420e15a5e.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.


Landcover data downloaded
Trying for date 2018-01-01


INFO:ibmpairs.paw:PAIRS query JSON initialized.
INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1664856000_32694394
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1664856000_32694394_ae23be732599e122042dfc81a5e5cf82.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.


Landcover data downloaded
Trying for date 2019-01-01


INFO:ibmpairs.paw:PAIRS query JSON initialized.
INFO:ibmpairs.paw:Query successfully submitted, reference ID: 1664856000_32822585
INFO:ibmpairs.paw:Here we go, PAIRS query result successfully downloaded as '1664856000_32822585_11ce3bbd8d50c476404c249c2554d7d6.zip'.
INFO:ibmpairs.paw:Data acknowledgement successfully loaded, print with `self.print_data_acknowledgement()`
INFO:ibmpairs.paw:PAIRS meta data loaded from 'output.info'.


Landcover data downloaded


# ------------
# Old code

In [ ]:
vpd_layers = [
        {     
            #Kelvin
            "type" : "raster", "id" : "49423",
            "aggregation" : "Mean",
            "alias": "temperature",
            "output" : "false"
        },
        {            
            #Kelvin
            "type" : "raster", "id" : "49422",
            "aggregation" : "Mean",
            "alias": "dewpoint",
            "output" : "false"
        },
        {
            #convert kelvin to rankine
            "alias": "temp_rankine",
            "expression": "$temperature * 1.8"
            
        },
        {
            #convert dewpoint kelvin to celcius
            "alias": "dewpoint_celcius",
            "expression": "$dewpoint - 273.15"
            
        },
        {
            #calculate saturation
            "alias": "saturation",
            "expression": " math:exp(( (math:pow(10,4) * -1.0440397) / $temp_rankine) + -11.29465 + (math:pow(10,-2) * -2.7022355 * $temp_rankine) + (math:pow(10,-5) * 1.289036 * math:pow($temp_rankine,2)) + (math:pow(10,-9) * -2.4780681 * math:pow($temp_rankine,3)) + (6.5459673 * math:log($temp_rankine))) "
        },
        {
            #convert kelvin to celcius
            "alias": "temp_celcius",
            "expression": "$temperature - 273.15"
            
        },
        {
            #calculate relative humidty
            "alias": "relative_humidity",
            "expression": "100 * (math:exp((17.625 * $dewpoint_celcius) / (243.04 + $dewpoint_celcius)) / (math:exp((17.625 * $temp_celcius) /(243.04 + $temp_celcius))))"
            
        },
        {
            #calculate partial presurre
            "alias": "partial_pressure",
            "expression": "$saturation * ($relative_humidity / 100) "
            
        },
        {
            #calculate vapour pressure deficit
            "alias": "vpd",
            "expression": "$saturation * (1-($relative_humidity /100))"
            #or, vpd = $saturation - $partial_pressure
            
        }
]

In [ ]:
# Using Beni's method (approach 2)

# this doesn't work because for specific humidity you need to specify a dimension (aka altitude)

vpd_layers = [
        {
            #convert kelvin to celcius
            "alias": "tmaxc",
            "expression": "$MaxT - 273.15", 
            "output" : "false"
        },
        {            
            # Mean specific humidity 
            "type" : "raster", "id" : "51513",
            "aggregation" : "Mean",
            "alias": "specifichum",
            "output" : "false"
        },
        {
            #Pressure
            "type" : "raster", "id" : "49439",
            "aggregation" : "Mean",
            "alias": "pressure",
            "output" : "false"
        },
        {
            # Calculate the mass mixing ratio of water vapor to dry air 
            "alias": "wair",
            "expression": "$specifichum / (1 - $specifichum) ",
            "output" : "false"
        },
        {
            #Water vapor pressure
            "alias": "eact",
            "expression": "$pressure * $wair * (8.3143/18.02) / ( (8.3143/28.963) + $wair * (8.3143/18.02))"#,
            #"output" : "false"
        },
        {
            # calculate saturation water vapour pressure in Pa
            "alias": "esat",
            "expression": "611*math:exp((17.27 * $tmaxc)/($tmaxc + 237.3))",
            "output" : "false"
        },
        {
            # Calculate vpd
            "alias": "vpd",
            "expression": "$esat - $eact",
            "output" : "false"
        },
        {
            # Calculate vpd
            "alias": "vpd_final",
            "expression": "math:max(0, $vpd)"
        }
]